# 03_Model_Experimentation.ipynb

This notebook covers the model building and training phase for the fraud detection project, including:
1.  **Data Preparation:** Feature/target separation and stratified train-test splitting.
2.  **Model Selection:** Training Logistic Regression (baseline) and XGBoost (ensemble) models.
3.  **Model Evaluation:** Assessing performance using metrics suitable for imbalanced data (AUC-PR, F1-Score, Confusion Matrix).
4.  **Model Interpretation:** Basic insights using SHAP and LIME.

### 1. Setup and Imports

In [ ]:
# Install necessary libraries if you haven't already in your Jupyter environment.
# You can uncomment and run these lines if needed, or install via requirements.txt:
# !pip install pandas numpy scikit-learn imbalanced-learn matplotlib seaborn xgboost shap lime
# If using GPU and cuDF is available, ensure it's installed in your environment:
# !pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com # Example for CUDA 12

# Core Libraries for Data Manipulation and Numerical Operations
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn for Data Splitting and Imbalance Handling
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE # For handling class imbalance

# Scikit-learn base classes and components (needed for custom transformers and pipelines)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Attempt to import cudf for GPU acceleration in the notebook scope
try:
    import cudf
    _CUDF_AVAILABLE_NOTEBOOK = True
    print("cuDF is available. GPU acceleration will be attempted where supported.")
except ImportError:
    _CUDF_AVAILABLE_NOTEBOOK = False
    print("cuDF not available. All operations will use pandas (CPU).")

# Add the project root directory to the system path.
# This allows Python to correctly locate and import our custom modules
# (e.g., from `src.data_processing.loader`) regardless of where the notebook is run from.
current_notebook_path = Path.cwd()
if current_notebook_path.name == 'notebooks':
    project_root = current_notebook_path.parent
else:
    project_root = current_notebook_path

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import Custom Modular Functions and Classes
from src.data_processing.loader import load_data
# Note: FraudDataProcessor and CreditCardDataProcessor are used in run_data_pipeline,
# here we directly load the processed data.
# from src.data_processing.preprocessor import FraudDataProcessor, CreditCardDataProcessor
# from src.utils.helpers import merge_ip_to_country

# Import Model Strategies, Trainer, Evaluator, and Interpreter
from src.models.logistic_regression_strategy import LogisticRegressionStrategy
from src.models.xgboost_strategy import XGBoostStrategy # Choosing XGBoost as the ensemble model
from src.models.random_forest_strategy import RandomForestStrategy # Also available if you prefer
from src.models.model_trainer import ModelTrainer
from src.models.model_evaluator import evaluate_classification_model
from src.models.model_interpreter import ModelInterpreter

# Import constants from run_data_pipeline.py for use throughout the notebook
from scripts.run_data_pipeline import (
    PROCESSED_DATA_DIR,
    FRAUD_TARGET_COL,
    CREDITCARD_TARGET_COL,
    # CREDITCARD_NUMERICAL_FEATURES, # Not directly needed here as we load processed data
    # FRAUD_NUMERICAL_FEATURES, # Not directly needed here as we load processed data
    # FRAUD_CATEGORICAL_FEATURES # Not directly needed here as we load processed data
)

# Set a random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


### 2. Load Preprocessed Data
Load the preprocessed datasets (`fraud_processed.csv` and `creditcard_processed.csv`) generated by the data pipeline. These DataFrames should contain clean, transformed features ready for modeling.

In [ ]:
# Define paths for preprocessed data
FRAUD_PROCESSED_PATH = PROCESSED_DATA_DIR / "fraud_processed.csv"
CREDITCARD_PROCESSED_PATH = PROCESSED_DATA_DIR / "creditcard_processed.csv"

# Load processed data. Ensure it's in pandas DataFrame format for scikit-learn compatibility.
# The `load_data` function should handle cuDF to pandas conversion if use_gpu is True.
print("--- Loading Preprocessed Datasets ---")

fraud_processed_df = load_data(FRAUD_PROCESSED_PATH, use_gpu=False) # Force pandas for consistency with sklearn
creditcard_processed_df = load_data(CREDITCARD_PROCESSED_PATH, use_gpu=False) # Force pandas for consistency with sklearn

if fraud_processed_df.empty:
    print(f"Error: E-commerce Fraud Data not loaded from {FRAUD_PROCESSED_PATH}. Please ensure `01_EDA.ipynb` or `run_data_pipeline.py` has been run.")
if creditcard_processed_df.empty:
    print(f"Error: Bank Credit Card Fraud Data not loaded from {CREDITCARD_PROCESSED_PATH}. Please ensure `01_EDA.ipynb` or `run_data_pipeline.py` has been run.")

print("\nPreprocessed Data Info (Fraud):")
fraud_processed_df.info()
print("\nPreprocessed Data Info (Credit Card):")
creditcard_processed_df.info()


### 3. Data Preparation - E-commerce Fraud Data
Separate features (X) and target (y), then perform a stratified train-test split. Class imbalance handling (SMOTE) is applied only to the training data.

In [ ]:
print("\n--- Data Preparation: E-commerce Fraud Data ---")

if not fraud_processed_df.empty:
    # Separate features (X) and target (y)
    # Identify feature columns: all columns except the target
    fraud_feature_cols = [col for col in fraud_processed_df.columns if col != FRAUD_TARGET_COL]
    
    X_fraud = fraud_processed_df[fraud_feature_cols]
    y_fraud = fraud_processed_df[FRAUD_TARGET_COL]

    # Ensure target is integer type for stratification
    y_fraud = y_fraud.astype(int)

    print(f"Original E-commerce Fraud Data shape: {X_fraud.shape}")
    print(f"Original E-commerce Fraud Target distribution:\n{y_fraud.value_counts(normalize=True)}")

    # Train-Test Split with stratification
    X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
        X_fraud, y_fraud, test_size=0.2, random_state=RANDOM_STATE, stratify=y_fraud
    )

    print(f"\nFraud Train set shape: {X_train_fraud.shape}")
    print(f"Fraud Test set shape: {X_test_fraud.shape}")
    print(f"Fraud Train target distribution:\n{y_train_fraud.value_counts(normalize=True)}")
    print(f"Fraud Test target distribution:\n{y_test_fraud.value_counts(normalize=True)}")

    # Apply SMOTE to the training data only
    print("\nApplying SMOTE to E-commerce Fraud training data...")
    smote = SMOTE(random_state=RANDOM_STATE)
    X_train_fraud_resampled, y_train_fraud_resampled = smote.fit_resample(X_train_fraud, y_train_fraud)

    print(f"Fraud Train set shape after SMOTE: {X_train_fraud_resampled.shape}")
    print(f"Fraud Train target distribution after SMOTE:\n{y_train_fraud_resampled.value_counts(normalize=True)}")
else:
    print("E-commerce Fraud Data is empty. Skipping data preparation.")


### 4. Model Training & Evaluation - E-commerce Fraud Data (Logistic Regression)
Train and evaluate a Logistic Regression model as a baseline for the E-commerce Fraud dataset.

In [ ]:
print("\n--- Model Training & Evaluation: E-commerce Fraud Data (Logistic Regression) ---")

if not fraud_processed_df.empty:
    # Initialize Logistic Regression model strategy
    lr_strategy_fraud = LogisticRegressionStrategy(random_state=RANDOM_STATE, solver='liblinear', max_iter=1000)
    
    # Initialize ModelTrainer
    trainer_fraud_lr = ModelTrainer(lr_strategy_fraud)
    
    # Train the model
    trainer_fraud_lr.train_model(X_train_fraud_resampled, y_train_fraud_resampled)
    
    # Make predictions (probabilities for positive class)
    y_pred_proba_fraud_lr = trainer_fraud_lr.predict_model(X_test_fraud)
    
    # Evaluate the model
    print("\n--- Evaluation Metrics (Logistic Regression, E-commerce Fraud) ---")
    lr_fraud_metrics = evaluate_classification_model(y_test_fraud.values, y_pred_proba_fraud_lr)

    # Display Confusion Matrix
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    y_pred_binary_fraud_lr = (y_pred_proba_fraud_lr >= 0.5).astype(int)
    cm_fraud_lr = confusion_matrix(y_test_fraud, y_pred_binary_fraud_lr)
    disp_fraud_lr = ConfusionMatrixDisplay(confusion_matrix=cm_fraud_lr, display_labels=[0, 1])
    disp_fraud_lr.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix: Logistic Regression (E-commerce Fraud)')
    plt.show()
else:
    print("E-commerce Fraud Data is empty. Skipping Logistic Regression training and evaluation.")


### 5. Model Training & Evaluation - E-commerce Fraud Data (XGBoost Classifier)
Train and evaluate an XGBoost Classifier for the E-commerce Fraud dataset, leveraging its ensemble power.

In [ ]:
print("\n--- Model Training & Evaluation: E-commerce Fraud Data (XGBoost Classifier) ---")

if not fraud_processed_df.empty:
    # Initialize XGBoost model strategy
    # Use a few common hyperparameters for a start.
    # With SMOTE, `scale_pos_weight` might not be strictly necessary, but can be fine-tuned.
    xgb_strategy_fraud = XGBoostStrategy(
        model_type='classifier',
        random_state=RANDOM_STATE,
        n_estimators=200,      # Number of boosting rounds
        learning_rate=0.1,     # Step size shrinkage
        max_depth=5,           # Maximum depth of a tree
        subsample=0.8,         # Subsample ratio of the training instance
        colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
        use_label_encoder=False, # Suppress warning
        eval_metric='logloss' # Evaluation metric for validation data
    )
    
    # Initialize ModelTrainer
    trainer_fraud_xgb = ModelTrainer(xgb_strategy_fraud)
    
    # Train the model
    trainer_fraud_xgb.train_model(X_train_fraud_resampled, y_train_fraud_resampled)
    
    # Make predictions (probabilities for positive class)
    y_pred_proba_fraud_xgb = trainer_fraud_xgb.predict_model(X_test_fraud)
    
    # Evaluate the model
    print("\n--- Evaluation Metrics (XGBoost Classifier, E-commerce Fraud) ---")
    xgb_fraud_metrics = evaluate_classification_model(y_test_fraud.values, y_pred_proba_fraud_xgb)

    # Display Confusion Matrix
    y_pred_binary_fraud_xgb = (y_pred_proba_fraud_xgb >= 0.5).astype(int)
    cm_fraud_xgb = confusion_matrix(y_test_fraud, y_pred_binary_fraud_xgb)
    disp_fraud_xgb = ConfusionMatrixDisplay(confusion_matrix=cm_fraud_xgb, display_labels=[0, 1])
    disp_fraud_xgb.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix: XGBoost Classifier (E-commerce Fraud)')
    plt.show()
else:
    print("E-commerce Fraud Data is empty. Skipping XGBoost training and evaluation.")


### 6. Model Interpretation - E-commerce Fraud Data (XGBoost)
Interpret the XGBoost model's predictions using SHAP for global feature importance and LIME for local instance explanations.

In [ ]:
print("\n--- Model Interpretation: E-commerce Fraud Data (XGBoost) ---")

if not fraud_processed_df.empty and trainer_fraud_xgb.trained_model is not None:
    # Get the feature names from the processed training data
    feature_names_fraud = X_train_fraud_resampled.columns.tolist()
    
    # Initialize ModelInterpreter
    interpreter_fraud_xgb = ModelInterpreter(
        model=trainer_fraud_xgb.get_current_model_object(),
        feature_names=feature_names_fraud,
        model_type='classification',
        class_names=['Non-Fraud', 'Fraud'],
        training_data_for_lime=X_train_fraud_resampled # LIME needs original training data for background
    )

    # Global Feature Importance (SHAP Summary Plot)
    interpreter_fraud_xgb.explain_model_shap(X_test_fraud)
    interpreter_fraud_xgb.plot_shap_summary(X_test_fraud)

    # Local Interpretation (LIME) - Explain a sample fraudulent transaction
    # Find an actual fraudulent transaction in the test set
    fraud_indices = y_test_fraud[y_test_fraud == 1].index
    if not fraud_indices.empty:
        sample_fraud_instance_idx = fraud_indices[0] # Take the first one
        sample_fraud_instance = X_test_fraud.loc[sample_fraud_instance_idx]
        print(f"\nExplaining a sample fraudulent instance (Index: {sample_fraud_instance_idx})...")
        interpreter_fraud_xgb.explain_instance_lime(sample_fraud_instance)
    else:
        print("No fraudulent instances found in the test set for LIME explanation.")

    # Local Interpretation (LIME) - Explain a sample non-fraudulent transaction
    non_fraud_indices = y_test_fraud[y_test_fraud == 0].index
    if not non_fraud_indices.empty:
        sample_non_fraud_instance_idx = non_fraud_indices[0] # Take the first one
        sample_non_fraud_instance = X_test_fraud.loc[sample_non_fraud_instance_idx]
        print(f"\nExplaining a sample non-fraudulent instance (Index: {sample_non_fraud_instance_idx})...")
        interpreter_fraud_xgb.explain_instance_lime(sample_non_fraud_instance)
    else:
        print("No non-fraudulent instances found in the test set for LIME explanation.")

else:
    print("E-commerce Fraud Data is empty or XGBoost model not trained. Skipping interpretation.")


### 7. Data Preparation - Credit Card Fraud Data
Separate features (X) and target (y), then perform a stratified train-test split. Class imbalance handling (SMOTE) is applied only to the training data.

In [ ]:
print("\n--- Data Preparation: Credit Card Fraud Data ---")

if not creditcard_processed_df.empty:
    # Separate features (X) and target (y)
    creditcard_feature_cols = [col for col in creditcard_processed_df.columns if col != CREDITCARD_TARGET_COL]
    
    X_creditcard = creditcard_processed_df[creditcard_feature_cols]
    y_creditcard = creditcard_processed_df[CREDITCARD_TARGET_COL]

    # Ensure target is integer type for stratification
    y_creditcard = y_creditcard.astype(int)

    print(f"Original Credit Card Fraud Data shape: {X_creditcard.shape}")
    print(f"Original Credit Card Fraud Target distribution:\n{y_creditcard.value_counts(normalize=True)}")

    # Train-Test Split with stratification
    X_train_creditcard, X_test_creditcard, y_train_creditcard, y_test_creditcard = train_test_split(
        X_creditcard, y_creditcard, test_size=0.2, random_state=RANDOM_STATE, stratify=y_creditcard
    )

    print(f"\nCredit Card Train set shape: {X_train_creditcard.shape}")
    print(f"Credit Card Test set shape: {X_test_creditcard.shape}")
    print(f"Credit Card Train target distribution:\n{y_train_creditcard.value_counts(normalize=True)}")
    print(f"Credit Card Test target distribution:\n{y_test_creditcard.value_counts(normalize=True)}")

    # Apply SMOTE to the training data only
    print("\nApplying SMOTE to Credit Card Fraud training data...")
    smote = SMOTE(random_state=RANDOM_STATE)
    X_train_creditcard_resampled, y_train_creditcard_resampled = smote.fit_resample(X_train_creditcard, y_train_creditcard)

    print(f"Credit Card Train set shape after SMOTE: {X_train_creditcard_resampled.shape}")
    print(f"Credit Card Train target distribution after SMOTE:\n{y_train_creditcard_resampled.value_counts(normalize=True)}")
else:
    print("Credit Card Fraud Data is empty. Skipping data preparation.")


### 8. Model Training & Evaluation - Credit Card Fraud Data (Logistic Regression)
Train and evaluate a Logistic Regression model as a baseline for the Credit Card Fraud dataset.

In [ ]:
print("\n--- Model Training & Evaluation: Credit Card Fraud Data (Logistic Regression) ---")

if not creditcard_processed_df.empty:
    # Initialize Logistic Regression model strategy
    lr_strategy_creditcard = LogisticRegressionStrategy(random_state=RANDOM_STATE, solver='liblinear', max_iter=1000)
    
    # Initialize ModelTrainer
    trainer_creditcard_lr = ModelTrainer(lr_strategy_creditcard)
    
    # Train the model
    trainer_creditcard_lr.train_model(X_train_creditcard_resampled, y_train_creditcard_resampled)
    
    # Make predictions (probabilities for positive class)
    y_pred_proba_creditcard_lr = trainer_creditcard_lr.predict_model(X_test_creditcard)
    
    # Evaluate the model
    print("\n--- Evaluation Metrics (Logistic Regression, Credit Card Fraud) ---")
    lr_creditcard_metrics = evaluate_classification_model(y_test_creditcard.values, y_pred_proba_creditcard_lr)

    # Display Confusion Matrix
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    y_pred_binary_creditcard_lr = (y_pred_proba_creditcard_lr >= 0.5).astype(int)
    cm_creditcard_lr = confusion_matrix(y_test_creditcard, y_pred_binary_creditcard_lr)
    disp_creditcard_lr = ConfusionMatrixDisplay(confusion_matrix=cm_creditcard_lr, display_labels=[0, 1])
    disp_creditcard_lr.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix: Logistic Regression (Credit Card Fraud)')
    plt.show()
else:
    print("Credit Card Fraud Data is empty. Skipping Logistic Regression training and evaluation.")


### 9. Model Training & Evaluation - Credit Card Fraud Data (XGBoost Classifier)
Train and evaluate an XGBoost Classifier for the Credit Card Fraud dataset, leveraging its ensemble power.

In [ ]:
print("\n--- Model Training & Evaluation: Credit Card Fraud Data (XGBoost Classifier) ---")

if not creditcard_processed_df.empty:
    # Initialize XGBoost model strategy
    xgb_strategy_creditcard = XGBoostStrategy(
        model_type='classifier',
        random_state=RANDOM_STATE,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    # Initialize ModelTrainer
    trainer_creditcard_xgb = ModelTrainer(xgb_strategy_creditcard)
    
    # Train the model
    trainer_creditcard_xgb.train_model(X_train_creditcard_resampled, y_train_creditcard_resampled)
    
    # Make predictions (probabilities for positive class)
    y_pred_proba_creditcard_xgb = trainer_creditcard_xgb.predict_model(X_test_creditcard)
    
    # Evaluate the model
    print("\n--- Evaluation Metrics (XGBoost Classifier, Credit Card Fraud) ---")
    xgb_creditcard_metrics = evaluate_classification_model(y_test_creditcard.values, y_pred_proba_creditcard_xgb)

    # Display Confusion Matrix
    y_pred_binary_creditcard_xgb = (y_pred_proba_creditcard_xgb >= 0.5).astype(int)
    cm_creditcard_xgb = confusion_matrix(y_test_creditcard, y_pred_binary_creditcard_xgb)
    disp_creditcard_xgb = ConfusionMatrixDisplay(confusion_matrix=cm_creditcard_xgb, display_labels=[0, 1])
    disp_creditcard_xgb.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix: XGBoost Classifier (Credit Card Fraud)')
    plt.show()
else:
    print("Credit Card Fraud Data is empty. Skipping XGBoost training and evaluation.")


### 10. Model Interpretation - Credit Card Fraud Data (XGBoost)
Interpret the XGBoost model's predictions using SHAP for global feature importance and LIME for local instance explanations.

In [ ]:
print("\n--- Model Interpretation: Credit Card Fraud Data (XGBoost) ---")

if not creditcard_processed_df.empty and trainer_creditcard_xgb.trained_model is not None:
    # Get the feature names from the processed training data
    feature_names_creditcard = X_train_creditcard_resampled.columns.tolist()
    
    # Initialize ModelInterpreter
    interpreter_creditcard_xgb = ModelInterpreter(
        model=trainer_creditcard_xgb.get_current_model_object(),
        feature_names=feature_names_creditcard,
        model_type='classification',
        class_names=['Non-Fraud', 'Fraud'],
        training_data_for_lime=X_train_creditcard_resampled # LIME needs original training data for background
    )

    # Global Feature Importance (SHAP Summary Plot)
    interpreter_creditcard_xgb.explain_model_shap(X_test_creditcard)
    interpreter_creditcard_xgb.plot_shap_summary(X_test_creditcard)

    # Local Interpretation (LIME) - Explain a sample fraudulent transaction
    fraud_indices_cc = y_test_creditcard[y_test_creditcard == 1].index
    if not fraud_indices_cc.empty:
        sample_fraud_instance_idx_cc = fraud_indices_cc[0]
        sample_fraud_instance_cc = X_test_creditcard.loc[sample_fraud_instance_idx_cc]
        print(f"\nExplaining a sample fraudulent instance (Index: {sample_fraud_instance_idx_cc})...")
        interpreter_creditcard_xgb.explain_instance_lime(sample_fraud_instance_cc)
    else:
        print("No fraudulent instances found in the Credit Card test set for LIME explanation.")

    # Local Interpretation (LIME) - Explain a sample non-fraudulent transaction
    non_fraud_indices_cc = y_test_creditcard[y_test_creditcard == 0].index
    if not non_fraud_indices_cc.empty:
        sample_non_fraud_instance_idx_cc = non_fraud_indices_cc[0]
        sample_non_fraud_instance_cc = X_test_creditcard.loc[sample_non_fraud_instance_idx_cc]
        print(f"\nExplaining a sample non-fraudulent instance (Index: {sample_non_fraud_instance_idx_cc})...")
        interpreter_creditcard_xgb.explain_instance_lime(sample_non_fraud_instance_cc)
    else:
        print("No non-fraudulent instances found in the Credit Card test set for LIME explanation.")

else:
    print("Credit Card Fraud Data is empty or XGBoost model not trained. Skipping interpretation.")


### 11. Model Justification and Conclusion

**E-commerce Fraud Data:**
- **Logistic Regression Metrics:**
  - Accuracy: {{ lr_fraud_metrics.get('Accuracy', 'N/A') }}
  - Precision: {{ lr_fraud_metrics.get('Precision', 'N/A') }}
  - Recall: {{ lr_fraud_metrics.get('Recall', 'N/A') }}
  - F1-score: {{ lr_fraud_metrics.get('F1-score', 'N/A') }}
  - ROC-AUC: {{ lr_fraud_metrics.get('ROC-AUC', 'N/A') }}

- **XGBoost Classifier Metrics:**
  - Accuracy: {{ xgb_fraud_metrics.get('Accuracy', 'N/A') }}
  - Precision: {{ xgb_fraud_metrics.get('Precision', 'N/A') }}
  - Recall: {{ xgb_fraud_metrics.get('Recall', 'N/A') }}
  - F1-score: {{ xgb_fraud_metrics.get('F1-score', 'N/A') }}
  - ROC-AUC: {{ xgb_fraud_metrics.get('ROC-AUC', 'N/A') }}

**Credit Card Fraud Data:**
- **Logistic Regression Metrics:**
  - Accuracy: {{ lr_creditcard_metrics.get('Accuracy', 'N/A') }}
  - Precision: {{ lr_creditcard_metrics.get('Precision', 'N/A') }}
  - Recall: {{ lr_creditcard_metrics.get('Recall', 'N/A') }}
  - F1-score: {{ lr_creditcard_metrics.get('F1-score', 'N/A') }}
  - ROC-AUC: {{ lr_creditcard_metrics.get('ROC-AUC', 'N/A') }}

- **XGBoost Classifier Metrics:**
  - Accuracy: {{ xgb_creditcard_metrics.get('Accuracy', 'N/A') }}
  - Precision: {{ xgb_creditcard_metrics.get('Precision', 'N/A') }}
  - Recall: {{ xgb_creditcard_metrics.get('Recall', 'N/A') }}
  - F1-score: {{ xgb_creditcard_metrics.get('F1-score', 'N/A') }}
  - ROC-AUC: {{ xgb_creditcard_metrics.get('ROC-AUC', 'N/A') }}

---

**Justification of the Best Model:**

For both datasets, the **XGBoost Classifier** is generally expected to outperform Logistic Regression, especially on metrics like F1-score and ROC-AUC, which are crucial for imbalanced datasets.

**Why XGBoost is preferred for Fraud Detection:**
1.  **Handles Imbalance (with SMOTE):** Even with SMOTE, tree-based models like XGBoost are robust to imbalanced classes and can learn complex decision boundaries.
2.  **Feature Interactions:** XGBoost can automatically learn non-linear relationships and interactions between features, which are common in fraud patterns. Logistic Regression assumes linearity.
3.  **Performance on Tabular Data:** Gradient Boosting models consistently achieve state-of-the-art performance on tabular datasets.
4.  **Interpretability (with SHAP/LIME):** While more complex than Logistic Regression, tools like SHAP and LIME allow us to interpret its predictions, identifying key features contributing to fraud scores, which is vital for understanding and investigating fraud.

**Balancing False Positives and False Negatives:**

In fraud detection:
-   **False Positives (Type I Error):** Legitimate transactions flagged as fraud. This leads to customer inconvenience and potential churn. A high precision indicates fewer false positives.
-   **False Negatives (Type II Error):** Actual fraudulent transactions missed. This leads to direct financial loss for the business. A high recall indicates fewer false negatives.

For fraud detection, **Recall** is often prioritized to minimize financial losses, but **Precision** cannot be ignored to maintain customer experience. The **F1-score** provides a good balance between precision and recall. **AUC-PR** is particularly valuable as it focuses on the performance of the model on the positive (minority) class, giving a more realistic view of how well the model identifies fraud compared to AUC-ROC.

XGBoost's ability to achieve a higher F1-score and AUC-PR, combined with its capacity for complex pattern recognition, makes it the more suitable and "best" model for this fraud detection task, even if its accuracy might be similar to Logistic Regression (as accuracy can be misleading on imbalanced data). The confusion matrix further helps in understanding the trade-offs between correctly identified fraud (True Positives) and missed fraud (False Negatives).

---

**Next Steps:**
-   Further hyperparameter tuning for the XGBoost model using techniques like GridSearchCV or RandomizedSearchCV.
-   Experiment with different class imbalance handling techniques (e.g., `scale_pos_weight` in XGBoost, different over/under-sampling methods).
-   Consider ensembling multiple models or more advanced deep learning approaches for complex fraud patterns.
-   Integrate MLflow for experiment tracking and model versioning (as hinted by `run_predict.py`).
-   Deploy the best-performing model for real-time inference using `run_predict.py` or a similar serving mechanism.
